# Google Play and Apple: Profitable Apps

<br></br>
<i>[Google Play](https://play.google.com/) is Google's digital distribution platform for the Android OS. Besides apps, it also serves as the company's digital media distribution service, offering music, books, television shows and movies. The [the App Store](https://www.apple.com/ios/app-store/) is Apple app distribution platform, which is indepedent from iTunes Store, Apple's distribution service for digital media.</i>

## Overview
<br>
<i>Note: Since this is the first exercise project in Dataquest's curriculum, this project involves just using basic Pythonic concepts to analyze data. The use of advanced concepts, and libraries are assumed to not be a part of the toolkit.</i>

A study needs to be conducted to effectively profile an app to be developed by company XYZ. This company's main revenue source are in-app ads, so the product needs to be free to download and install. Because ad revenue is mostly influenced by user count, the more users who see and engage with the ads, the better. To help developers determine the type of app to build, the focus is on mobile app data from Google Play and the App Store.

## Dataset

As of September 2018, there are a little over 4 million iOS and Android apps available for download. Exploring the data in this scale requires significant resources, so analysis is done on sample subsets from the Google Play data and the App Store data. They are as follows:

- [A set containing data about ten thousand Android apps;](https://www.kaggle.com/lava18/google-play-store-apps) 
- [A set containing data about seven thousand iOS apps.](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps)

In [1]:
from csv import reader

# Google Play
android_file = reader(open('datasets/googleplaystore.csv'))
android_list = list(android_file)
android_header = android_list[0]
android = android_list[1:]

# App Store
ios_file = reader(open('datasets/applestore.csv'))
ios_list = list(ios_file)
ios_header = ios_list[0]
ios = ios_list[1:]

To explore these two data sets, the function `explore_data` provides a way to take a sample and display some metadata. Below are samples of the Google Play data and the App Store data, with row and column counts.

In [2]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n')

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))
        
print(android_header)
print('\n')
explore_data(android, 0, 3, True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


Number of rows: 10841
Number of columns: 13


In [3]:
print(ios_header)
print('\n')
explore_data(ios, 0, 3, True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Number of rows: 7197
Number of columns: 16


Using the samples above, these are the things to note:

- The data sets include exactly 10,841 Android apps, and 7,197 iOS apps.
- At a quick glance, the Google Play data columns that might be useful for analysis are `App`, `Category`, `Reviews`, `Installs`, `Type`, `Price`, and `Genres`.
- Using the iOS data set [documentation](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home) as guide, the columns that seem relevant are: `track_name`, `currency`, `price`, `rating_count_tot`, `rating_count_ver`, and `prime_genre`.



## Data Cleaning

Prior to analysis, it seems useful to go through the data, and take out any errors or inconsistencies. Duplicates, false entries, or other inaccuracies may lead to issues later on in the analysis.

To start, in [one of the discussions](https://www.kaggle.com/lava18/google-play-store-apps/discussion/66015) from the source of the Google Play data set, there seems to be a consensus about an error on row 10472.

In [4]:
print(android_header)
print(android[10472])

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


The rating for this row, `19` seems inaccurate, since the maximum Google Play app rating is 5. As consequence, this row would be removed.

In [5]:
# before and after row counts for verification
print(len(android))
del android[10472]
print(len(android))

10841
10840


### Checking for Duplicates

When analyzing data, it is important that duplicates do not skew results, so duplicate entries need to be consolidated. Dupe tests on both Google Play and iOS follow.

In [6]:
duplicate_apps = []
unique_apps = []

for app in ios:
    name = app[1]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
    
print('Number of duplicate apps:', len(duplicate_apps))
print('Examples of duplicate apps:', duplicate_apps[:15])

Number of duplicate apps: 2
Examples of duplicate apps: ['Mannequin Challenge', 'VR Roller Coaster']


In [7]:
print(ios_header)
for app in ios:
    name = app[1]
    if name == 'VR Roller Coaster':
        print(app)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']
['952877179', 'VR Roller Coaster', '169523200', 'USD', '0.0', '107', '102', '3.5', '3.5', '2.0.0', '4+', 'Games', '37', '5', '1', '1']
['1089824278', 'VR Roller Coaster', '240964608', 'USD', '0.0', '67', '44', '3.5', '4.0', '0.81', '4+', 'Games', '38', '0', '1', '1']


These rows seem to refer to two different builds / versions of the same app. Since the more recent build would include more current reviews, the newer builds would be kept.

In [8]:
duplicate_apps = []
unique_apps = []

for app in android:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
    
print('Number of duplicate apps:', len(duplicate_apps))
print('Examples of duplicate apps:', duplicate_apps[:15])

Number of duplicate apps: 1181
Examples of duplicate apps: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack', 'FreshBooks Classic', 'Insightly CRM', 'QuickBooks Accounting: Invoicing & Expenses', 'HipChat - Chat Built for Teams', 'Xero Accounting Software']


In [9]:
print(android_header)
for app in android:
    name = app[0]
    if name == 'Slack':
        print(app)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
['Slack', 'BUSINESS', '4.4', '51507', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'August 2, 2018', 'Varies with device', 'Varies with device']
['Slack', 'BUSINESS', '4.4', '51507', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'August 2, 2018', 'Varies with device', 'Varies with device']
['Slack', 'BUSINESS', '4.4', '51510', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'August 2, 2018', 'Varies with device', 'Varies with device']


The rows seem to be the same, except for the fourth column (corresponding to the review count). Since a higher review count usually corresponds to a more reliable rating, the column with the highest review count would be retained.

### Removing Duplicates

Although dropping the duplicate rows in the iOS data set seem straightforward, deleting about 1,100 rows from the Android data set would be cumbersome without some form of automation. The identified steps to resolve this are as follows:

1. Create a dictionary where each key is a unique app name, and the value is the highest number of reviews of that app
2. Use the dictionary to create a new data set, which would have only one entry per app

In [10]:
reviews_max = {}

for app in android:
    name = app[0]
    n_reviews = float(app[3])
    
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
        
    elif name not in reviews_max:
        reviews_max[name] = n_reviews

print('Expected Row Count:', len(android) - 1181) # minus duplicates
print('Actual Row Count:', len(reviews_max))

Expected Row Count: 9659
Actual Row Count: 9659


In [11]:
android_clean = []
already_added = []

for app in android:
    name = app[0]
    n_reviews = float(app[3])
    
    if (reviews_max[name] == n_reviews) and (name not in already_added):
        android_clean.append(app)
        already_added.append(name)
        
explore_data(android_clean, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9659
Number of columns: 13


In [12]:
ios_header[5]

'rating_count_tot'

In [13]:
reviews_max = {}

for app in ios:
    name = app[1]
    n_reviews = float(app[5])
    
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
        
    elif name not in reviews_max:
        reviews_max[name] = n_reviews

print('Expected Row Count:', len(ios) - 2) # minus duplicates
print('Actual Row Count:', len(reviews_max))

Expected Row Count: 7195
Actual Row Count: 7195


In [14]:
ios_clean = []
already_added = []

for app in ios:
    name = app[1]
    n_reviews = float(app[5])
    
    if (reviews_max[name] == n_reviews) and (name not in already_added):
        ios_clean.append(app)
        already_added.append(name)
        
explore_data(ios_clean, 0, 3, True)

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Number of rows: 7195
Number of columns: 16


### Removing Non-English Apps

Since the preliminary target market is US-based users, it seems useful to remove some of the apps that may not be directed toward an English-speaking audience. Below are some example apps, from both sets:

In [15]:
print('iOS Examples:')
print(ios_clean[813][1])
print(ios_clean[6731][1])

iOS Examples:
爱奇艺PPS -《欢乐颂2》电视剧热播
激ムズ！和のひとふで書き！ 〜頭をつかう脳トレパズルゲーム〜


In [16]:
print('Android Examples:')
print(android_clean[4412][0])
print(android_clean[7940][0])

Android Examples:
中国語 AQリスニング
لعبة تقدر تربح DZ


One way to go about this is to identify app names with symbols not commonly used in English texts. 

Since English characters are encoded using the ASCII standard, non-ASCII characters could be used as a filter. In addition, because each ASCII character corresponds to a number between 0 and 127, the built-in `ord()` function could be used as a delimiter.

The only other consideration is that some English app names use emojis or other symbols (™, — (em dash), – (en dash), etc.) that fall outside of the ASCII range. To minimize the impact of data loss, an app would only be removed if the name has more than three non-ASCII characters. 

This approach would not be perfect, but this seems good enough at this point of the analysis.

In [17]:
def is_english(string):
    non_ascii = 0
    
    for character in string:
        if ord(character) > 127:
            non_ascii += 1
    
    if non_ascii > 3:
        return False
    else:
        return True

print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(is_english('Instagram'))
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))

False
True
True
True


In [18]:
android_english = []
ios_english = []

for app in android_clean:
    name = app[0]
    if is_english(name):
        android_english.append(app)
        
for app in ios_clean:
    name = app[1]
    if is_english(name):
        ios_english.append(app)
        
explore_data(android_english, 0, 3, True)
print('\n')
explore_data(ios_english, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9614
Number of columns: 13


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 

### Removing Paid Apps

Since the analysis should be on free apps, paid apps should be removed from the study.

In [19]:
android_final = []
ios_final = []

for app in android_english:
    price = app[7]
    if price == '0':
        android_final.append(app)
        
for app in ios_english:
    price = app[4]
    if price == '0.0':
        ios_final.append(app)
        
print(len(android_final))
print(len(ios_final))

8864
3220


## Analysis

As mentioned in the introduction, the aim is to determine the kinds of free apps likely to attract a significant user base, which is correlated to higher company revenue. 

To minimize risks and overhead, validation strategy for the app is comprised of three steps:

1. Build an MVP Android app, and add it to Google Play.
2. If the app has a good response from users, develop it further.
3. If the app is profitable after six months, build an iOS version of the app and launch it on the App Store.

Because the end goal is to make the app available on both App Store and Google Play, the app profile has to be successful on both markets. 

### Popular Genres: App Counts

The most common genres for each market could be a good place to start for some high-level analysis. To start, the `prime_genre` column of the App Store data set, and the `Category` columns of the Google Play data set would be used to build frequency tables.

In [20]:
def freq_table(dataset, index):
    table = {}
    total = 0
    
    for row in dataset:
        total += 1
        value = row[index]
        if value in table:
            table[value] += 1
        else:
            table[value] = 1
    
    table_percentages = {}

    for key in table:
        percentage = (table[key] / total) * 100
        table_percentages[key] = round(percentage, 2)
    
    return table_percentages


def sort_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)
        
    table_sorted = sorted(table_display, reverse = True)
    
    for entry in table_sorted:
        print('{0:5.2f}%: {1}'.format(entry[0],entry[1]).title())

In [21]:
sort_table(ios_final, -5)

58.14%: Games
 7.89%: Entertainment
 4.97%: Photo & Video
 3.66%: Education
 3.29%: Social Networking
 2.61%: Shopping
 2.52%: Utilities
 2.14%: Sports
 2.05%: Music
 2.02%: Health & Fitness
 1.74%: Productivity
 1.58%: Lifestyle
 1.34%: News
 1.24%: Travel
 1.12%: Finance
 0.87%: Weather
 0.81%: Food & Drink
 0.56%: Reference
 0.53%: Business
 0.43%: Book
 0.19%: Navigation
 0.19%: Medical
 0.12%: Catalogs


In [22]:
sort_table(android_final, 1)

18.91%: Family
 9.72%: Game
 8.46%: Tools
 4.59%: Business
 3.90%: Lifestyle
 3.89%: Productivity
 3.70%: Finance
 3.53%: Medical
 3.40%: Sports
 3.32%: Personalization
 3.24%: Communication
 3.08%: Health_And_Fitness
 2.94%: Photography
 2.80%: News_And_Magazines
 2.66%: Social
 2.34%: Travel_And_Local
 2.25%: Shopping
 2.14%: Books_And_Reference
 1.86%: Dating
 1.79%: Video_Players
 1.40%: Maps_And_Navigation
 1.24%: Food_And_Drink
 1.16%: Education
 0.96%: Entertainment
 0.94%: Libraries_And_Demo
 0.93%: Auto_And_Vehicles
 0.82%: House_And_Home
 0.80%: Weather
 0.71%: Events
 0.65%: Parenting
 0.64%: Art_And_Design
 0.62%: Comics
 0.60%: Beauty


The data above suggests that the App Store is dominated by apps designed for fun (games, entertainment, etc.) having about 3 in 4 apps released. Games occupy 58%, Entertainment close to 8%, and Photo and Video about 5%, making the top three genres. Apps that are more utilitarian (Education, for instance) occupy significantly less of the share. However, the fact that there are a lot of games available does not imply that there are a lot of users; the demand might not be proportional to the supply.

The landscape seems a bit different on Google Play, where there are more apps designed for practical purposes. Although the top two categories are for fun (Family is mostly about games for kids), utilitarian apps make up about 65 - 70% of the total.

### Popular Genres: User Count

Another way to find out the most popular genres — and one that seems closer to goal — is to look at the user base, through number of installs. For the Google Play data set, we can find this information in the `Installs` column, but for the App Store data set this information is missing. The `rating_count_tot`, which counts the user ratings, could serve as proxy.

### *User Counts: App Store*

In [23]:
ios_genres = freq_table(ios_final, -5)
ios_ratings = {}

for genre in ios_genres:
    sum_ratings = 0
    sum_apps = 0
    
    for app in ios_final:
        app_genre = app[-5]
        if app_genre == genre:            
            ratings = float(app[5])
            sum_ratings += ratings
            sum_apps += 1

    avg_ratings = round(sum_ratings / sum_apps)
    ios_ratings[avg_ratings] = genre
    
ios_sorted = sorted(ios_ratings, reverse=True)
    
for i in ios_sorted:
    print('{0:6,}: {1}'.format(i, ios_ratings[i]))

86,090: Navigation
74,942: Reference
71,548: Social Networking
57,327: Music
52,280: Weather
39,758: Book
33,334: Food & Drink
31,468: Finance
28,442: Photo & Video
28,244: Travel
26,920: Shopping
23,298: Health & Fitness
23,009: Sports
22,813: Games
21,248: News
21,028: Productivity
18,684: Utilities
16,486: Lifestyle
14,030: Entertainment
 7,491: Business
 7,004: Education
 4,004: Catalogs
   612: Medical


In [24]:
def display_top_apps(genre):
    top_10 = {}
    count = 0
    
    for app in ios_final: 
        name = app[1]
        ratings = int(app[5])
        if count < 20 and app[-5] == genre:
            top_10[ratings] = name
            count += 1
            
    for k in top_10:
        print('{0:10,}: {1}'.format(k,top_10[k]))
    
display_top_apps('Navigation')

   345,046: Waze - GPS Navigation, Maps & Real-time Traffic
   154,911: Google Maps - Navigation & Transit
    12,811: Geocaching®
     3,582: CoPilot GPS – Car Navigation & Offline Maps
       187: ImmobilienScout24: Real Estate Search in Germany
         5: Railway Route Search


In [25]:
display_top_apps('Reference')

   985,920: Bible
   200,047: Dictionary.com Dictionary & Thesaurus
    54,175: Dictionary.com Dictionary & Thesaurus for iPad
    26,786: Google Translate
    18,418: Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran
    17,588: New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition
    16,849: Merriam-Webster Dictionary
    12,122: Night Sky
     8,535: City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE)
     4,693: LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools
     1,497: GUNS MODS for Minecraft PC Edition - Mods Tools
       826: Guides for Pokémon GO - Pokemon GO News and Cheats
       762: WWDC
       718: Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free
        14: VPN Express
         8: Real Bike Traffic Rider Virtual Reality Glasses
         0: Jishokun-Japanese English Dictionary & Translator


In [26]:
display_top_apps('Social Networking')

 2,974,676: Facebook
 1,061,624: Pinterest
   373,519: Skype for iPhone
   351,466: Messenger
   334,293: Tumblr
   287,589: WhatsApp Messenger
   260,965: Kik
   177,501: ooVoo – Free Video Call, Text and Voice
   164,963: TextNow - Unlimited Text + Calls
   164,249: Viber Messenger – Text & Call
   112,778: Followers - Social Analytics For Instagram
    97,072: MeetMe - Chat and Meet New People
    90,414: We Heart It - Fashion, wallpapers, quotes, tattoos
    85,535: InsTrack for Instagram - Analytics Plus More
    75,412: Tango - Free Video Call, Voice and Chat
    71,856: LinkedIn
    60,659: Match™ - #1 Dating App.
    60,163: Skype for iPad
    52,642: POF - Best Dating App for Conversations
    49,510: Timehop


In [27]:
display_top_apps('Music')

 1,126,879: Pandora - Music & Radio
   878,563: Spotify Music
   402,925: Shazam - Discover music, artists, videos & lyrics
   293,228: iHeartRadio – Free Music & Radio Stations
   135,744: SoundCloud - Music & Audio
   131,695: Magic Piano by Smule
   119,316: Smule Sing!
   110,420: TuneIn Radio - MLB NBA Audiobooks Podcasts Music
   106,235: Amazon Music
    82,602: SoundHound Song Search & Music Player
    48,905: Sonos Controller
    30,845: Bandsintown Concerts
    28,606: Karaoke - Sing Karaoke, Unlimited Songs!
    26,286: My Mixtapez Music
    26,227: Sing Karaoke Songs Unlimited with StarMaker
    25,403: Ringtones for iPhone & Ringtone Maker
    25,193: Musi - Unlimited Music For YouTube
    18,202: AutoRap by Smule
    15,053: Spinrilla - Mixtapes For Free
    14,268: Napster - Top Music & Radio


Although navigation apps have the highest average user reviews, it has been heavily influenced by almost half a million user reviews for Waze and Google Maps. The same pull applies to social networking apps, where average numbers are driven by Facebook, Pinterest, Skype, and other social media giants. This is also true for music apps, with Pandora, Spotify, and Shazam.

Reference apps have 74,942 user ratings on average, but it's actually the Bible and Dictionary.com which skew up the average rating. However, this niche seems to show some potential.


### *User Counts: Google Play*

For the Google Play market, although there is data about the number of installs, the numbers don't seem precise enough. Most values are open-ended (100+, 1,000+, 5,000+, etc.):

One thing we could do is take another popular book and turn it into an app where we could add different features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes about the book, etc. On top of that, we could also embed a dictionary within the app, so users don't need to exit our app to look up words in an external app.

This idea seems to fit well with the fact that the App Store is dominated by for-fun apps. This suggests the market might be a bit saturated with for-fun apps, which means a practical app might have more of a chance to stand out among the huge number of apps on the App Store.

Other genres that seem popular include weather, book, food and drink, or finance. The book genre seem to overlap a bit with the app idea we described above, but the other genres don't seem too interesting to us:

- Weather apps — people generally don't spend too much time in-app, and the chances of making profit from in-app adds are low. Also, getting reliable live weather data may require us to connect our apps to non-free APIs.

- Food and drink — examples here include Starbucks, Dunkin' Donuts, McDonald's, etc. So making a popular food and drink app requires actual cooking and a delivery service, which is outside the scope of our company.

- Finance apps — these apps involve banking, paying bills, money transfer, etc. Building a finance app requires domain knowledge, and we don't want to hire a finance expert just to build an app.

Now let's analyze the Google Play market a bit.

In [28]:
sort_table(android_final, 5)

15.73%: 1,000,000+
11.55%: 100,000+
10.55%: 10,000,000+
10.20%: 10,000+
 8.39%: 1,000+
 6.92%: 100+
 6.83%: 5,000,000+
 5.56%: 500,000+
 4.77%: 50,000+
 4.51%: 5,000+
 3.54%: 10+
 3.25%: 500+
 2.30%: 50,000,000+
 2.13%: 100,000,000+
 1.92%: 50+
 0.79%: 5+
 0.51%: 1+
 0.27%: 500,000,000+
 0.23%: 1,000,000,000+
 0.05%: 0+
 0.01%: 0


One problem with this data is precision, since the values are estimates. An app with 100,000+ installs may have 100,000, 200,000, or 350,000 installs. The only consolation is that a high level of precision is not necessary to determine which app genres attract the most users.

To proceed, the minimum count per value will be assumed. An app with 100,000+ installs has 100,000 installs, and an app with 1,000,000+ installs has 1,000,000 installs, and so on.

In [29]:
android_categories = freq_table(android_final, 1)
android_cats = {}

for category in android_categories:
    total_installs = 0
    len_category = 0
    
    for app in android_final:
        app_category = app[1]
        if app_category == category:            
            installs = app[5]
            installs = installs.replace(',', '')
            installs = installs.replace('+', '')
            total_installs += float(installs)
            len_category += 1
    
    avg_installs = round(total_installs / len_category)
    android_cats[avg_installs] = category

android_sorted = sorted(android_cats, reverse=True)

for i in android_sorted:
    x = android_cats[i].replace('_', ' ')
    x = x.replace(' AND ', ' & ').title()
    print('{0:10,}: {1}'.format(i, x))

38,456,119: Communication
24,727,872: Video Players
23,253,652: Social
17,840,110: Photography
16,787,331: Productivity
15,588,016: Game
13,984,078: Travel & Local
11,640,706: Entertainment
10,801,391: Tools
 9,549,178: News & Magazines
 8,767,812: Books & Reference
 7,036,877: Shopping
 5,201,483: Personalization
 5,074,486: Weather
 4,188,822: Health & Fitness
 4,056,942: Maps & Navigation
 3,695,642: Family
 3,638,640: Sports
 1,986,335: Art & Design
 1,924,898: Food & Drink
 1,833,495: Education
 1,712,290: Business
 1,437,816: Lifestyle
 1,387,692: Finance
 1,331,541: House & Home
   854,029: Dating
   817,657: Comics
   647,318: Auto & Vehicles
   638,504: Libraries & Demo
   542,604: Parenting
   513,152: Beauty
   253,542: Events
   120,551: Medical


On average, communication apps have the most installs: 38,456,119. This number is heavily skewed up by a few apps that have over one billion installs (WhatsApp, Facebook Messenger, Skype, Google Chrome, Gmail, and Hangouts), and a few others with over 100 and 500 million installs:

In [30]:
huge_installs = ['1,000,000,000+', '500,000,000+', '100,000,000+']

for count in huge_installs:
    print('Apps with {0} installs:'.format(count))
    for app in android_final:
        app_name = app[0]
        category = app[1]
        installs = app[5]

        if category == 'COMMUNICATION' and installs == count:
            print('- ', app_name)

Apps with 1,000,000,000+ installs:
-  WhatsApp Messenger
-  Messenger – Text and Video Chat for Free
-  Skype - free IM & video calls
-  Google Chrome: Fast & Secure
-  Gmail
-  Hangouts
Apps with 500,000,000+ installs:
-  Google Duo - High Quality Video Calls
-  imo free video calls and chat
-  LINE: Free Calls & Messages
-  UC Browser - Fast Download Private & Secure
-  Viber Messenger
Apps with 100,000,000+ installs:
-  imo beta free calls and text
-  Android Messages
-  Who
-  GO SMS Pro - Messenger, Free Themes, Emoji
-  Firefox Browser fast & private
-  Messenger Lite: Free Calls & Messages
-  Kik
-  KakaoTalk: Free Calls & Text
-  Opera Mini - fast web browser
-  Opera Browser: Fast and Secure
-  Telegram
-  Truecaller: Caller ID, SMS spam blocking & Dialer
-  UC Browser Mini -Tiny Fast Private & Secure
-  WeChat
-  Yahoo Mail – Stay Organized
-  BBM - Free Calls & Messages


In [31]:
def under_100m_avg(genre):

    under_100_m = []
    for app in android_final:
        n_installs = app[5]
        n_installs = n_installs.replace(',', '')
        n_installs = n_installs.replace('+', '')
        if (app[1] == genre) and (float(n_installs) < 100000000):
            under_100_m.append(float(n_installs))

    rounded = round(sum(under_100_m) / len(under_100_m))
    print('{:10,}'.format(rounded))

under_100m_avg('COMMUNICATION')

 3,603,485


In [32]:
under_100m_avg('VIDEO_PLAYERS')

 5,544,878


If communication apps with over 100 million installs were removed from the consideration, the average user base would be a tenth of its current count. The same reduction applies to video players, where the market is dominated by apps like Youtube, Google Play Movies & TV, or MX Player. This pattern is repeated for social apps (with Facebook, Instagram, Google+, etc.), photography apps (Google Photos), or productivity apps (Microsoft Word, Dropbox, Google Calendar, Evernote, etc.).

Again, the main concern is that certain genres might appear more popular than they really are, due to the heavy pull of some giants. This also implies that these giants already have a monopoly of the market, and that competing with these apps might be tough. In the same vein, the games genre already seems saturated, so profitability may be difficult to accomplish.

### App Profile: Book and Reference Apps

The books and reference genre looks fairly popular, but doesn't seem to have significant competition. It's interesting to explore this in more depth.

Below are some apps and their number of installs:

In [33]:
tally_app_count = 0

for app in android_final:
    if app[1] == 'BOOKS_AND_REFERENCE' and tally_app_count < 20 :
        tally_app_count += 1
        print(app[0], ':', app[5])

E-Book Read - Read Book for free : 50,000+
Download free book with green book : 100,000+
Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Free Panda Radio Music : 100,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
English Grammar Complete Handbook : 500,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
Google Play Books : 1,000,000,000+
AlReader -any text book reader : 5,000,000+
Offline English Dictionary : 100,000+
Offline: English to Tagalog Dictionary : 500,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
Recipes of Prophetic Medicine for free : 500,000+
ReadEra – free ebook reader : 1,000,000+
Anonymous caller detection : 10,000+
Ebook Reader : 5,000,000+
Litnet - E-books : 100,000+


In [34]:
for app in android_final:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000,000+'
                                         or app[5] == '500,000,000+'
                                         or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

Google Play Books : 1,000,000,000+
Bible : 100,000,000+
Amazon Kindle : 100,000,000+
Wattpad 📖 Free Books : 100,000,000+
Audiobooks from Audible : 100,000,000+


In [35]:
for app in android_final:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000+'
                                         or app[5] == '5,000,000+'
                                         or app[5] == '10,000,000+'
                                         or app[5] == '50,000,000+'):
        print(app[0], ':', app[5])

Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
AlReader -any text book reader : 5,000,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
ReadEra – free ebook reader : 1,000,000+
Ebook Reader : 5,000,000+
Read books online : 5,000,000+
eBoox: book reader fb2 epub zip : 1,000,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
Moon+ Reader : 10,000,000+
English-Myanmar Dictionary : 1,000,000+
Golden Dictionary (EN-AR) : 1,000,000+
All Language Translator Free : 1,000,000+
Aldiko Book Reader : 10,000,000+
Dictionary - WordWeb : 5,000,000+
50000 Free eBooks & Free AudioBooks : 5,000,000+
Al-Quran (Free) : 10,000,000+
Al Quran Indonesia : 10,000,000+
Al'Quran Bahasa Indonesia : 10,000,000+
Al Quran Al karim : 1,000,000+
Al Quran : EAlim - Translations & MP3 Offline : 5,000,000+
Koran Read &MP3 30 Juz Offline : 1,000,000+
H

## Conclusion

Apps for eBooks, libraries, and dictionaries dominate the 'Book and Reference' genre, so avoiding these top types is logical. Looking at the second tier of popular apps, the few apps build around Quran — even the app build around the Bible — suggests that building an app around a popular book can be profitable. 

It seems that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets. To distinguish the app from competition, it may be necessary to add special features, such as: daily quotes from the book, an audiobook companion, quizzes or interactive lessons, a in-app forum, etc.